### 2 layer NN, fully connected, 3*preamble_length node layers, nn.sigmoid - output layer identity preamble_length = 100, channel size = 2, learning rate = 0.0001, decay = 1-10^-9
### Generate training data: channels and approximate channel inverses 
### Generate test data: channels and approximate channel inverses

In [45]:
import numpy as np
import scipy.signal as sig
from numpy import linalg as LA
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# Create training and test data for the NN

# number of random channels that will be trained and tested on
num_train=500000# 1000000
num_test=10000
SNR = 10

# assume we know the channel_length
channel_length = 2
num_preambles = 1
preamble_length = 100
recieved_length = preamble_length + channel_length - 1

preambles = np.random.randint(0,2,(num_preambles,preamble_length)) 

def add_awgn_noise(signal,SNR_dB):
    """  Adds AWGN noise vector to signal 
         to generate a resulting signal vector y of specified SNR in dB
    """
    L=len(signal)
    SNR = 10**(SNR_dB/10.0) #SNR to linear scale
    Esym=np.sum(np.square(np.abs(signal)))/L #Calculate actual symbol energy
    N0=Esym/SNR; #Find the noise spectral density
    if(isinstance(signal[0], complex)):
        noiseSigma=np.sqrt(N0/2.0)#Standard deviation for AWGN Noise when x is complex
        n = noiseSigma*(np.random.randn(1,L)+1j*np.random.randn(1,L))#computed noise 
    else:
        noiseSigma = np.sqrt(N0);#Standard deviation for AWGN Noise when x is real
        n = noiseSigma*np.random.randn(1,L)#computed noise
    y = signal + n #received signal
    return signal

# channel_train: assume we are working with just real parts, list of all used channels
# preamble_train: use same channel on num_preambles different preambles and store them to train with: 
#               [actual preamble, convolved preamble of same size]
channel_train = np.zeros((num_train,channel_length))
preamble_train = np.zeros((num_train*num_preambles, preamble_length*2))

channel_test = np.zeros((num_test,channel_length))
preamble_test = np.zeros((num_test*num_preambles, preamble_length*2))

In [46]:
# Create data with training data to be [preamble, convolved preamble]

for i in range(0, num_train):
    channel_train[i,:]=np.random.uniform(0.2,1,channel_length)
    # if the total power is greater than 1, then normalize
    if sum(channel_train[i])>=1:
        channel_train[i] = channel_train[i]/(sum(channel_train[i]))
        
    for k in range(num_preambles):
        preamble_conv = add_awgn_noise(sig.convolve(preambles[k], channel_train[i], mode='same'), SNR)
        preamble_total = np.hstack((preambles[k], preamble_conv))
        preamble_train[i+k,:] = preamble_total
        
        
for i in range(0, num_test):
    channel_test[i,:]=np.random.uniform(0.2,1,channel_length)
    # if the total power is greater than 1, then normalize
    if sum(channel_test[i])>=1:
        channel_test[i] = channel_test[i]/(sum(channel_test[i]))
        
    for k in range(num_preambles):
        preamble_conv = add_awgn_noise(sig.convolve(preambles[k], channel_test[i], mode='same'), SNR)
        preamble_total = np.hstack((preambles[k], preamble_conv))
        preamble_test[i+k,:] = preamble_total


In [ ]:
# fix a random seed to have the same results
np.random.seed()

learning_rate = 0.0001#0.0001
epochs = 5000#10000
batch_size = 1000
test_averaging=100
decay = 1 - 10*1e-9#1 - 10*1e-10

# placeholders for input and output
adaptive_learning_rate = tf.placeholder_with_default(learning_rate, [])
recieved_preamble = tf.placeholder(tf.float32, [None, preamble_length*2])
channel_real = tf.placeholder(tf.float32, [None, channel_length])

layer1 = tf.contrib.layers.fully_connected(recieved_preamble, num_outputs=3*preamble_length, activation_fn=tf.nn.sigmoid)
#layer2 = tf.contrib.layers.fully_connected(layer1, num_outputs=3*preamble_length, activation_fn=tf.nn.sigmoid)
layer3 = tf.contrib.layers.fully_connected(layer1, num_outputs=channel_length, activation_fn=tf.identity)

channel_estimate = layer3

cost_fn = tf.reduce_mean(tf.reduce_mean((channel_real-channel_estimate)**2, axis=1))
optimizer = tf.train.AdamOptimizer(learning_rate=adaptive_learning_rate).minimize(cost_fn)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
plt.figure()

test_costs =[]
test_channels = []
train_costs = []
train_channels =[]

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        
        # for each batch start the batch with passing the first preamble through the identity channel
        identity_channel = np.zeros(channel_length)
        identity_channel[0] = 1
        preamble_conv_id = sig.convolve(preambles[0], identity_channel, mode='same')
        preamble_batch = np.hstack((preambles[0], preamble_conv_id))
        
        real_channel_batch = identity_channel
        
        for rand in range(0,batch_size-1):
            rand_int = np.random.randint(0,num_train*num_preambles)
            preamble_batch = np.vstack((preamble_batch, preamble_train[rand_int]))
            real_channel_batch = np.vstack((real_channel_batch, channel_train[rand_int]))
            
        preamble_batch.reshape((batch_size, preamble_length*2))
        real_channel_batch.reshape((batch_size, channel_length))
        
        _,cost,channel_est = sess.run(
            [optimizer, cost_fn, channel_estimate], 
            feed_dict={recieved_preamble: preamble_batch, 
                       channel_real: real_channel_batch ,adaptive_learning_rate: learning_rate * (decay**epoch)})
        
        train_costs.append(cost)
        train_channels.append(channel_est)
        
        if epoch % 100 == 0: 
            plt.plot(epoch, cost, 'bo')
            mc_cost, mc_channel = sess.run(
                [cost_fn, channel_estimate], feed_dict={recieved_preamble: preamble_test, channel_real: channel_test})
            print('Epoch {}, Cost {}, Test Cost: {}'.format(epoch, cost, mc_cost))
            test_costs.append(mc_cost)
            test_channels.append(mc_channel)
            
    saved_model = saver.save(sess, 'my-test-model')

Epoch 0, Cost 0.07882671803236008, Test Cost: 0.061335645616054535
Epoch 100, Cost 0.003442976390942931, Test Cost: 0.003212160198017955
Epoch 200, Cost 0.0004624091961886734, Test Cost: 0.00040186592377722263
Epoch 300, Cost 0.00010318632121197879, Test Cost: 9.614410373615101e-05
Epoch 400, Cost 2.1521826056414284e-05, Test Cost: 2.1823434508405626e-05
Epoch 500, Cost 4.73438103654189e-06, Test Cost: 4.7219918997143395e-06
Epoch 600, Cost 2.062858584395144e-06, Test Cost: 1.8171667761635035e-06
Epoch 700, Cost 1.6486612821609015e-06, Test Cost: 1.4314725831354735e-06
Epoch 800, Cost 1.6430464029326686e-06, Test Cost: 1.3892458810005337e-06
Epoch 900, Cost 1.511118171038106e-06, Test Cost: 1.3750285461355816e-06
Epoch 1000, Cost 1.5112727851374075e-06, Test Cost: 1.369095002701215e-06
Epoch 1100, Cost 1.5150959598031477e-06, Test Cost: 1.3766421034233645e-06
Epoch 1200, Cost 1.5132731050471193e-06, Test Cost: 1.3552768223235034e-06
Epoch 1300, Cost 1.4055441397431423e-06, Test Cost: 1

In [ ]:
plt.plot(np.log10(train_costs))
plt.title("Log Cost on Training set vs. Epochs")
plt.xlabel("Epochs")
plt.ylabel("Log Cost on Training set")

In [ ]:
plt.plot(np.log10(test_costs))
plt.title("Log Cost on Test set vs. 10's of epochs")
plt.xlabel("Number of Epochs in multiples of 10")
plt.ylabel("Log Cost on Test set")

In [ ]:
print(test_costs)

In [ ]:
print(np.log10(test_costs))

In [ ]:
print(channel_test[90:100])

In [ ]:
print(test_channels[49][90:100])